# Baseline Approach

This Notebook tries to implement the baseline approach described in the paper accompanind the Exam: 

- [Privacy-Preserving Recommender Systems Challenge on Twitter’s HomeTimeline]{https://arxiv.org/pdf/2004.13715.pdf}

The baseline approch is described in chapter 5.3

In [1]:
#import findspark
#findspark.init("/usr/spark-2.4.1")
import pyspark
from pyspark import SQLContext

import numpy as np

In [ ]:
# Set Spark Config
conf = SparkConf().setAppName("RecSys-Challenge-Baseline-Approach").setMaster("yarn")
conf = (conf.set("deploy-mode","cluster")
       .set("spark.driver.memory","100g")
       .set("spark.executor.memory","100g")
       .set("spark.driver.cores","1")
       .set("spark.num.executors","100")
       .set("spark.executor.cores","4")
       .set("spark.driver.maxResultSize", "100g"))
sc = pyspark.SparkContext(conf=conf)
sql = SQLContext(sc)


sc = pyspark.SparkContext()
sql = SQLContext(sc)

## Data Loading

Read Sample Data as Spark DataFrame and assign column names according to RecSys Challenge Documentation

In [ ]:
datafile = "hdfs:///user/e1553958/reysys/data/training_sample.tsv"

df = (sql.read
    .format("csv")
    .option("header", "false")
    .option("sep", "\x01")
    .load(datafile,  inferSchema="true")
    .toDF("text_tokens", "hashtags", "tweet_id", "present_media", "present_links", "present_domains","tweet_type", "language", "tweet_timestamp", "engaged_with_user_id", "engaged_with_user_follower_count","engaged_with_user_following_count", "engaged_with_user_is_verified", "engaged_with_user_account_creation",\
               "engaging_user_id", "engaging_user_follower_count", "engaging_user_following_count", "engaging_user_is_verified","engaging_user_account_creation", "engaged_follows_engaging", "reply_timestamp", "retweet_timestamp", "retweet_with_comment_timestamp", "like_timestamp"))

In [ ]:
df

In [ ]:
# Change the boolean variables to interger (0 and 1)
from pyspark.sql.functions import col, split, when, array
df = df.withColumn("engaged_with_user_is_verified",col("engaged_with_user_is_verified").cast("Integer"))
df = df.withColumn("engaging_user_is_verified",col("engaging_user_is_verified").cast("Integer"))
df = df.withColumn("engaged_follows_engaging",col("engaged_follows_engaging").cast("Integer"))

In [ ]:
# Split the string representations of lists
## Convert the text tokens to array of ints
split_text = pyspark.sql.functions.split(df['text_tokens'], '\t')
df = df.withColumn("text_tokens", split_text)

## Convert present media to array of strings
split_text = pyspark.sql.functions.split(df['present_media'], '\t')
df = df.withColumn("present_media", when(col('present_media').isNull(), array().cast("array<string>")).otherwise(split_text))

## Convert present links to array of strings
split_text = pyspark.sql.functions.split(df['present_links'], '\t')
df = df.withColumn("present_links", when(col('present_links').isNull(), array().cast("array<string>")).otherwise(split_text))

## Convert hashtags to array of strings
split_text = pyspark.sql.functions.split(df['hashtags'], '\t')
df = df.withColumn("hashtags", when(col('hashtags').isNull(), array().cast("array<string>")).otherwise(split_text))

## Convert present_domains to array of strings
split_text = pyspark.sql.functions.split(df['present_domains'], '\t')
df = df.withColumn("present_domains", when(col('present_domains').isNull(), array().cast("array<string>")).otherwise(split_text))


In [ ]:
df.take(1)

In [ ]:
numeric_cols = ['engaged_with_user_follower_count', 
                'engaged_with_user_following_count', 
                'engaged_with_user_account_creation',
                'engaging_user_follower_count', 
                'engaging_user_following_count',
                'engaging_user_account_creation',
                'tweet_timestamp',
               ]


categorical_cols = ['tweet_type', 'language', 
                    'engaged_with_user_is_verified', 'engaging_user_is_verified', 'engaged_follows_engaging']

id_cols = ['tweet_id', 'engaged_with_user_id', 'engaging_user_id']

response_cols = ['reply_timestamp', 
                 'retweet_timestamp',
                 'retweet_with_comment_timestamp', 
                 'like_timestamp'
                ]

tweet_feature_cols = ['text_tokens', 'hashtags', 'present_media', 'present_links', 'present_domains']

### Helper Functions

In [ ]:
from pyspark.ml.feature import QuantileDiscretizer, StringIndexer, FeatureHasher, OneHotEncoderEstimator, CountVectorizer,PCA, VectorAssembler

nq = 50 # number of quantiles to use

def create_quantilesDiscretizer(input_col: str) -> QuantileDiscretizer:
    """
    Create a Quantile Discretizer for a specified column 
    Uses as output colum the input + _discretized
    
    Parameters
    ----------
    input_col: str
        Name of the Input Column
        
    Return
    ------
    QuantileDiscretizer
    """
    output_col = input_col + "_discretized"
    return QuantileDiscretizer(numBuckets=nq,
                                  relativeError=0.,
                                  handleInvalid='keep',
                                  inputCol=input_col,
                                  outputCol=output_col)

def create_stringIndexer(input_col):
    """
    Create a String Indexer for a specified column 
    Uses as output colum the input + _indexed
    
    Parameters
    ----------
    input_col: str
        Name of the Input Column
        
    Return
    ------
    StringIndexer
    """
    output_col = input_col + "_indexed"
    return StringIndexer(inputCol=input_col,
                         outputCol=output_col,
                        handleInvalid='keep',)


def create_featureHasher(input_col):
    """
    Create a Feature Hasher for a specified column 
    Uses as output colum the input + _oneHot (creates oneHotEncodings for strings)
    
    Parameters
    ----------
    input_col: str
        Name of the Input Column
        
    Return
    ------
    FeatureHasher
    """
    output_col = input_col + "_oneHot"
    return FeatureHasher(numFeatures=nq,
                         inputCols=[input_col],
                         outputCol=output_col)


def create_countVectorizer(input_col):
    output_col = input_col + "_vectorized"
    return CountVectorizer(inputCol=input_col,
                           outputCol=output_col)


#### Create Transformer

In [ ]:
# Encode Numeric Features (5.3.1)
quantile_discretizers_numeric = [ create_quantilesDiscretizer(col) for col in numeric_cols ]

# Encode Categorical Features (5.3.2)
string_indexer_categorical = [ create_stringIndexer(col) for col in categorical_cols]

# Encode ID Features (5.3.3)
id_feature_hashers = [ create_featureHasher(col) for col in id_cols]

# Encode Tweet Features (5.3.4 + 5.3.5)
tweet_countVectorizers = [ create_countVectorizer(col) for col in tweet_feature_cols]


# One-Hot-Encode Features
columns_to_encode = [ col+"_discretized" for col in numeric_cols]
columns_to_encode.extend([ col+"_indexed" for col in categorical_cols])
encoded_columns = [ col+"_oneHot" for col in numeric_cols]
encoded_columns.extend([ col+"_onHot" for col in categorical_cols])

onHotEncoder = OneHotEncoderEstimator(inputCols=columns_to_encode, 
                                      outputCols=encoded_columns,dropLast=False,handleInvalid="keep" )



# Add Vectors with VectorAssembler
encoded_columns.extend([ col+"_oneHot" for col in id_cols ])
num_cat_id_feature_assambler = VectorAssembler(inputCols=encoded_columns,
                                               outputCol="non_tweet_features")

tweet_features_encoded = [ col+"_vectorized" for col in tweet_feature_cols]
tweet_feature_assambler = VectorAssembler(inputCols=tweet_features_encoded,
                                               outputCol="tweet_features")

# Perform Dimensionality Reduction

#non_tweet_pca = PCA(k=16, 
#          inputCol="non_tweet_features", 
#          outputCol="non_tweet_features_reduced")

#tweet_pca = PCA(k=16, 
#          inputCol="tweet_features", 
#          outputCol="tweet_features_reduced")

features = ['non_tweet_features', 'tweet_features']
feature_assambler = VectorAssembler(inputCols=features,
                                               outputCol="features")

In [ ]:
from pyspark.ml import Pipeline


# create a list of all transformers
stages = list()
stages.extend(quantile_discretizers_numeric)
stages.extend(string_indexer_categorical)
stages.extend(id_feature_hashers)
stages.append(onHotEncoder)
stages.extend(tweet_countVectorizers)
stages.append(num_cat_id_feature_assambler)
stages.append(tweet_feature_assambler)
#stages.append(non_tweet_pca)
#stages.append(tweet_pca)
stages.append(feature_assambler)

pipeline = Pipeline(stages=stages)

In [ ]:
# fit and transform dataframe
df = pipeline.fit(df).transform(df)

In [ ]:
df.select('features').take(1)

#### Encode Label Columns 

Create on column with array of 0 and 1

In [ ]:
from pyspark.sql.functions import  lit, col

label_col = [ col(name) for name in response_cols]

def encode_response(x):
    return when(col(x).isNull(), float(0)).otherwise(float(1))

for column in response_cols:
    df = df.withColumn(column, encode_response(column))
    
# for now only use likes as target
df = df.withColumn("label", col("like_timestamp"))

In [ ]:
splits = df.select("features", "label").randomSplit([0.8, 0.2], seed=1234)
train = splits[0]
test = splits[1]

In [ ]:
# create the trainer and set its parameters
from pyspark.ml.classification import MultilayerPerceptronClassifier

layers = [93549, 128, 64, 34, 1]

trainer = MultilayerPerceptronClassifier(
  layers=layers,
  blockSize=128,
  seed=1234,
  maxIter=100)

# train the model
model = trainer.fit(train)

In [ ]:
# compute accuracy on the test set
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
val result = model.transform(test)
val predictionAndLabels = result.select("prediction", "label")
val evaluator = new MulticlassClassificationEvaluator()
  .setMetricName("accuracy")